In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
LEGISLICE_API_TOKEN = os.getenv("LEGISLICE_API_TOKEN")

In [2]:
from authorityspoke.io.downloads import LegisClient

CLIENT = LegisClient(api_token=LEGISLICE_API_TOKEN)

In [3]:
offense_statute = CLIENT.read("/us/usc/t18/s1960/a")

In [4]:
print(offense_statute)

"Whoever knowingly conducts, controls, manages, supervises, directs, or owns all or part of an unlicensed money transmitting business, shall be fined in accordance with this title or imprisoned not more than 5 years, or both." (/us/usc/t18/s1960/a 2013-07-18)


This example is based on [United States v. Harmon](https://www.courtlistener.com/opinion/4843029/united-states-v-harmon/), a case from the District Court of the District of Columbia

In [5]:
from authorityspoke import Entity, Fact, Holding, Predicate
no_license = Fact(
            "$business was licensed as a money transmitting business",
            truth=False,
            terms=Entity("Helix"),
        )
operated = Fact(
    "$person operated $business as a business",
    terms=[Entity("Harmon"), Entity("Helix")],
)
transmitting = Fact(
    "$business was a money transmitting business", terms=Entity("Helix")
)
offense = Fact(
    "$person committed the offense of conducting an unlicensed money transmitting business",
    terms=Entity("Harmon"),
)

In [6]:
offense_holding = Holding.from_factors(
    inputs=[operated, transmitting, no_license],
    outputs=offense,
    enactments=offense_statute,
    universal=True,
)

In [7]:
print(offense_holding)

the Holding to ACCEPT
  the Rule that the court MAY ALWAYS impose the
    RESULT:
      the fact that <Harmon> committed the offense of conducting an
      unlicensed money transmitting business
    GIVEN:
      the fact that <Harmon> operated <Helix> as a business
      the fact that <Helix> was a money transmitting business
      the fact it was false that <Helix> was licensed as a money
      transmitting business
    GIVEN the ENACTMENT:
      "Whoever knowingly conducts, controls, manages, supervises, directs, or owns all or part of an unlicensed money transmitting business, shall be fined in accordance with this title or imprisoned not more than 5 years, or both." (/us/usc/t18/s1960/a 2013-07-18)


In [8]:
definition_statute = CLIENT.read("/us/usc/t18/s1960/b/2")

In [9]:
definition_statute.text

'the term “money transmitting” includes transferring funds on behalf of the public by any and all means including but not limited to transfers within this country or to locations abroad by wire, check, draft, facsimile, or courier; and'

In [10]:
bitcoin = Fact(
    "$business transferred bitcoin on behalf of the public",
    terms=Entity("Helix"),
)
bitcoin_holding = Holding.from_factors(
    inputs=bitcoin,
    outputs=transmitting,
    enactments=definition_statute,
    universal=True,
)

The combined effect of the two holdings above is that if a person operated a business that transferred bitcoin on behalf of the public without a "money transmitting business" license, the person may be found guilty of the offense.

In [11]:
print(bitcoin_holding)

the Holding to ACCEPT
  the Rule that the court MAY ALWAYS impose the
    RESULT:
      the fact that <Helix> was a money transmitting business
    GIVEN:
      the fact that <Helix> transferred bitcoin on behalf of the public
    GIVEN the ENACTMENT:
      "the term “money transmitting” includes transferring funds on behalf of the public by any and all means including but not limited to transfers within this country or to locations abroad by wire, check, draft, facsimile, or courier; and" (/us/usc/t18/s1960/b/2 2013-07-18)


In [12]:
result = bitcoin_holding + offense_holding

Note that "the fact that `<Helix>` was a money transmitting business" is included as an output of the Holding, but is no longer needed as an input to trigger the Holding.

In [13]:
print(result)

the Holding to ACCEPT
  the Rule that the court MAY ALWAYS impose the
    RESULT:
      the fact that <Harmon> committed the offense of conducting an
      unlicensed money transmitting business
      the fact that <Helix> was a money transmitting business
    GIVEN:
      the fact that <Harmon> operated <Helix> as a business
      the fact it was false that <Helix> was licensed as a money
      transmitting business
      the fact that <Helix> transferred bitcoin on behalf of the public
    GIVEN the ENACTMENTS:
      "Whoever knowingly conducts, controls, manages, supervises, directs, or owns all or part of an unlicensed money transmitting business, shall be fined in accordance with this title or imprisoned not more than 5 years, or both." (/us/usc/t18/s1960/a 2013-07-18)
      "the term “money transmitting” includes transferring funds on behalf of the public by any and all means including but not limited to transfers within this country or to locations abroad by wire, check, draft, 

In [14]:
result_with_new_context = result.new_context(([Entity("Harmon"), Entity("Helix")], [Entity("Schrute"), Entity("Schrute Bucks")]))

In [15]:
print(result_with_new_context)

the Holding to ACCEPT
  the Rule that the court MAY ALWAYS impose the
    RESULT:
      the fact that <Schrute> committed the offense of conducting an
      unlicensed money transmitting business
      the fact that <Schrute Bucks> was a money transmitting business
    GIVEN:
      the fact that <Schrute> operated <Schrute Bucks> as a business
      the fact it was false that <Schrute Bucks> was licensed as a money
      transmitting business
      the fact that <Schrute Bucks> transferred bitcoin on behalf of the
      public
    GIVEN the ENACTMENTS:
      "Whoever knowingly conducts, controls, manages, supervises, directs, or owns all or part of an unlicensed money transmitting business, shall be fined in accordance with this title or imprisoned not more than 5 years, or both." (/us/usc/t18/s1960/a 2013-07-18)
      "the term “money transmitting” includes transferring funds on behalf of the public by any and all means including but not limited to transfers within this country or to 

https://www.courtlistener.com/opinion/175697/united-states-v-mazza-alaluf/
https://www.courtlistener.com/api/rest/v3/opinions/?id=175697
    

In [ ]:
new_york_anchor = "the trial evidence was sufficient to permit a finding that Mazza-Alaluf operated a money transmitting business in New York, Illinois, and Michigan that required appropriate state licenses."

new_york_offense = Fact(
    "$defendant used ${defendant}'s business $business to commit the New York offense "
    "of engaging in the business of receiving money "
    "for transmission or transmitting the same, without a license therefor",
    terms=[Entity("Mazza-Alaluf"), Entity("Turismo Costa Brava")])
definition_statute = CLIENT.read("/us/usc/t18/s1960/b/1/A")
no_appropriate_state_license = Fact(
            ("$defendant operated $business without an appropriate money transmitting "
            "license in a State where such operation was punishable as a misdemeanor "
            "or a felony under State law"),
            terms=[Entity("Mazza-Alaluf"), Entity("Turismo Costa Brava")],
        )

In [17]:
from authorityspoke import Entity, Fact, Holding, Predicate
no_license = Fact(
            ("$defendant operated $business without an appropriate money transmitting "
            "license in a State where such operation was punishable as a misdemeanor "
            "or a felony under State law"),
            terms=[Entity("Mazza-Alaluf"), Entity("Turismo Costa Brava")],
        )
operated = Fact(
    "$person operated $business as a business",
    terms=[Entity("Mazza-Alaluf"), Entity("Turismo Costa Brava")],
)
transmitting = Fact(
    "$business was a money transmitting business", terms=Entity("Turismo Costa Brava")
)
offense = Fact(
    "$person committed the offense of conducting a money transmitting business without a license required by state law",
    terms=Entity("Mazza-Alaluf"),
)
not_domestic = Fact(
"$business was a domestic financial institution", truth=False, terms=Entity("Turismo Costa Brava"))
definition_statute = CLIENT.read("/us/usc/t18/s1960/b/1/A")
domestic_institution_statute = CLIENT.read("/us/usc/t18/s5312/b/1")
no_domestic_office = Fact("$business lacked an agent, agency, branch, or office within the United States", terms=Entity("Turismo Costa Brava"))
holding = Holding.from_factors(
    inputs=[no_license, operated, transmitting],
    outputs=offense,
    despite=[not_domestic, no_domestic_office],
    enactments=definition_statute,
    enactments_despite=domestic_institution_statute
)
    
"""
Holding anchor:

Title 18 U.S.C. § 1960(b)(1)(A), which makes it a crime to operate a money transmitting 
business without appropriate state licenses, does not require the government to prove 
that the charged business was a "domestic financial institution," as *215 defined in 
those parts of Title 31 governing federal reporting requirements for such institutions.
"""

LegislicePathError: No enacted text found for query https://authorityspoke.com/api/v1/us/usc/t18/s5312/b/1/